In [1]:
import cv2
import numpy as np

#1
# flow 를 그리는 함수 생성
def drawFlow(img, flow, thresh=2, stride=8):
    h, w = img.shape[:2]
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1]) # 옵티컬플로우로부터 크기와 각도를 추출
    flow2 = np.int32(flow)
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            dx, dy = flow2[y, x]
            if mag[y, x] > thresh: # 크기가 일정 이상만 원과 선을 그림
                cv2.circle(img, (x, y), 2, (0,255,0), -1)
                cv2.line(img, (x, y), (x+dx, y+dy), (255,0,0),1)

In [2]:
import cv2
import numpy as np

#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
# 4개 변수 초기화, 마우스 이벤트에 사용

# mouse event 함수 정의
def onMouse(event, x, y, flags, param=None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y) # 마우스 드래그 포인트
        mouse_status = 1  # 1은 roi를 설정하기 시작했다는 의미
        tracking_start = False # 아직 트랙킹 안 한 상태
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2 # dragging
            # 마우스 시작포인트, 끝 포인트 즉 roi 영역 저장, 드래그 하는 중이라는 의미
            
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3 # complete 마우스에서 손을 떼었다라는 의미

In [3]:
#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
# 4개 변수 초기화, 마우스 이벤트에 사용

#2
def createTracker():
    tracker = cv2.TrackerKCF()
#     tracker = cv2.TrackerMIL_create()
#     tracker = cv2.TrackerGOTURN_create()
#     tracker = cv2.TrackerCSRT_create()
    return tracker

In [ ]:
#3
cv2.namedWindow('tracking') # tracking 이라는 윈도우 생성
cv2.setMouseCallback('tracking', onMouse) # onMous에 등록

cap = cv2.VideoCapture('image/highwayCCTV2.mp4') # 새로운 영상 읽기
if (not cap.isOpened()):
    print('Error opening video')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height, width), dtype=np.uint8)
tracker = None

#4
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    if mouse_status==2:                                                  # 마우스 드래그 중
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0,0), 2)
    
    if mouse_status==3:                                                  # 마우스 손 뗀 상태, roi 설정 완료
        print('initialize...')
        mouse_status = 0
        track_box = (roi[0], roi[1], roi[2]-roi[0], roi[3]-roi[1])
        if tracker != None:
#             tracker.clear()
            del tracker
        tracker = createTracker()
        tracking_start = tracker.init(frame, track_box)
        
    if tracking_start:                                                   # 트랙킹 시작되었다면,
        ret, track_box = tracker.update(frame)
        if ret: # track success
            x,y,w,h = track_box
            p1 = (int(x), int(y))
            p2 = (int(x+w), int(y+h))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    
    cv2.imshow('tracking', frame)                                        # meanShift 결과
    key = cv2.waitKey(25)
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)